# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [19]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [20]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/Users/edwinmartinez/DataModelingWithApacheCassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [21]:
full_data_rows_list = [] 

for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)       
        for line in csvreader:
            full_data_rows_list.append(line) 
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [22]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [23]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [24]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [26]:
'''
This query created a table that is meant to return the
artist, song title, and song length given sessionId and itemInSession.
As such, sessionId and itemInSession are the primary key
'''

query = "CREATE TABLE IF NOT EXISTS music_app_history "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [27]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        

#### Do a SELECT to verify that the data have been inserted into each table

In [28]:
query = "SELECT artist, song, length FROM music_app_history "
query = query + "WHERE sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [29]:
'''
This query creates a table that is meant to search for 
artist, song, first name, last name given userId and sessionId.
We use userId and sessionId plus itemInSession as primary key
'''
query = "CREATE TABLE IF NOT EXISTS music_app_history_user_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [30]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO music_app_history_user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


In [31]:
query = "SELECT artist, song, firstName, lastName FROM music_app_history_user_session \
         WHERE userId = 10 and sessionId = 182 \
         ORDER BY itemInSession desc"

try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

for row in rows:
    print(row)

Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')


In [32]:
'''
Here I created a table to return first and last name
given song name
the song and userId where the primary key.
'''
query = "CREATE TABLE IF NOT EXISTS music_app_history_specific_song \
        (song text, userId int, firstName text, lastName text, \
        PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history_specific_song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


In [ ]:
query = "SELECT firstName, lastName FROM music_app_history_specific_song \
         WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e) 

for row in rows:
    print(row)

### Drop the tables before closing out the sessions

In [ ]:
session.execute("DROP TABLE music_app_history")
session.execute("DROP TABLE music_app_history_user_session")
session.execute("DROP TABLE music_app_history_specific_song")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()